In [2]:
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'ticks')
import warnings
import os
import pickle
from io import StringIO
warnings.filterwarnings("ignore")
from anndata import AnnData
import anndata
from scipy import sparse
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import subprocess
from functools import partial

#__DATA READING__
def get_peak_id(data, keys = ['chr','start','end']):
    return data[keys[0]].astype(str) + '_' + data[keys[1]].astype(str).str.strip() + '_' + data[keys[2]].astype(str).str.strip()

def read_bias_file(filepath):
    colnames = 'chr,start,end,barcode,dup_group,bias1,bias2,gc_content,fragment_len,log_duprate,peak_chr,peak_start,peak_end,corrected_count'.split(',')
    fragments = pd.read_csv(filepath, header = None, sep = '\t')
    fragments.columns = colnames
    fragments['peak_id'] = get_peak_id(fragments, keys = ['peak_chr','peak_start','peak_end'])
    return fragments

def read_stats(filepath, columns):
    stats = pd.read_csv(filepath, sep = '\t', header = None)
    stats.columns = columns
    #stats['samples'] = stats.samples.apply(lambda x : list(map(float, x.strip('[]').split(', ')))).apply(np.array)
    return stats

def read_barcode_stats(filepath):
    return read_stats(filepath, ['barcode','mean_log_duprate','fragment_count','samples'])

def read_peak_stats(filepath):
    peak_stats = read_stats(filepath, ['peak_id','median_log_duprate','fragment_count','samples'])
    peak_stats['rank'] = peak_stats.median_log_duprate.rank(method = 'first',na_option = 'keep')
    return peak_stats

def read_sparse_countmatrix(barcode_file, peaks_file, count_matrix_file, min_peak_proportion = 0):

    barcode_stats = read_barcode_stats(barcode_file)
    peak_stats = read_peak_stats(peaks_file)
    
    counts = sparse.load_npz(count_matrix_file)

    data = AnnData(X = counts, obs = barcode_stats, var = peak_stats)

    data.var['accessible_in_cells'] =  np.array((data.X > 0).sum(axis=0)).reshape(-1)

    return data

#__PLOT GENERATION__
def benchmark_fragment_model(fragments, dupcounts, peak_stats):

    fragments = fragments.merge(peak_stats[['peak_id','accessible_in_cells']], on = 'peak_id', how = 'inner')\
        .merge(dupcounts, left_on = 'dup_group', right_on = 'group')
    fragments['true_log_duprate'] = np.log(fragments.duplicate_counts / fragments.accessible_in_cells)

    return fragments[['log_duprate', 'true_log_duprate']]

def sample_ranks(andata_chunk, num_samples = 10000):

    cluster_peak_counts = np.array(andata_chunk.X.sum(axis = 0)).reshape(-1)
    cluster_peak_probs = cluster_peak_counts / cluster_peak_counts.sum()

    peak_samples = np.random.choice(len(cluster_peak_probs), p = cluster_peak_probs, 
        size = (min(len(cluster_peak_probs), num_samples),))
    
    sampled_ranks = andata_chunk.var.iloc[peak_samples]['rank'].values.astype(int)

    return sampled_ranks

def get_bias_peak_enrichment(andata, cluster_key, num_samples = 10000):

    ranks_dict = {}
    for cluster_id in np.unique(andata.obs[cluster_key]):
        ranks_dict[cluster_id] = sample_ranks(
            andata[andata.obs[cluster_key] == cluster_id, ~np.isnan(andata.var['rank'])], 
            num_samples=num_samples
        )

    ranks_df = pd.DataFrame(ranks_dict)
    ranks_df = pd.melt(ranks_df, var_name = 'cluster_id', value_name = 'rank')
    return ranks_df

def get_fragment_distribution_by_peak_and_cluster(andata, peaks_str, cluster_key):

    selected_fragments = read_bias_file(StringIO(peaks_str))

    selected_fragments = selected_fragments.merge(andata.obs[['barcode',cluster_key]], on = 'barcode', how = 'right')\
        .merge(andata.var[['peak_id','rank']], on = 'peak_id', how = 'right')

    stratified_subset = selected_fragments.groupby(['peak_id',cluster_key], group_keys=False)\
        .apply(lambda x : x.sample(min(len(x), 150)))

    return stratified_subset

#__SCANPY ADDONS___
def get_differential_peaks(andata, top_n = 200):

    sc.tl.rank_genes_groups(andata, 'leiden', method='wilcoxon')

    return pd.DataFrame(andata.uns['rank_genes_groups']['names']).head(200)

def standardize(x):
    return np.clip((x - x.mean()) / x.std(), -3, 3)

def plot_umap(andata,*,color_key, key = 'X_umap', quantitative=True, ax = None, center=True, legend = False,
             hue_order = None, hue_norm = None):
    data = andata.obsm['X_umap']
    if type(color_key) == str:
        color_var = andata.obs[color_key].values
    else:
        color_var = color_key
        
    if not quantitative:
        ax = sns.scatterplot(x = data[:,0], y = data[:,1], size = 0.5, 
                       hue = color_var.astype('str'), 
                       hue_order = hue_order, legend = legend, ax = ax)
    else:
        ax = sns.scatterplot(x = data[:,0], y = data[:,1], hue = standardize(color_var) if center else color_var,
                   palette = "vlag", size = 0.5, legend = False, ax = ax, hue_norm = hue_norm)
    ax.set(xticklabels = [], xticks = [], yticks = [])
    sns.despine()
    return ax

def _binary_search(func, acceptible_range, low, high, iter_num = 0, max_iter = 10):

    midpoint = low + (high - low)/2

    output = func(midpoint)
    
    if (output >= acceptible_range[0] and output < acceptible_range[1]) or iter_num >= max_iter:
        return midpoint

    if output < acceptible_range[0]:
        return _binary_search(func, acceptible_range, midpoint, high, iter_num + 1, max_iter=max_iter)
    else:
        return _binary_search(func, acceptible_range, low, midpoint, iter_num+1, max_iter=max_iter)        
        

def get_num_clusters(andata, resolution):
    sc.tl.leiden(andata, resolution = 10**resolution)
    num_clusters = andata.obs.leiden.nunique()
    print('Resolution: {}, # Clusters: {}'.format(str(10**resolution), str(num_clusters)))
    return num_clusters
    
def process_counts(andata, resolution = 0.5, tune = False, num_clusters = (7,10)):
    
    lsi_model = TruncatedSVD(n_components=50)
    X_LSI = lsi_model.fit_transform(
        TfidfTransformer().fit_transform(andata.X)
    )

    andata.obsm['X_lsi'] = X_LSI
    
    sc.pp.neighbors(andata, use_rep = 'X_lsi')

    if tune:
        cluster_func = partial(get_num_clusters, andata)
        resolution = _binary_search(cluster_func, num_clusters, -2, 0)
    else:
        sc.tl.leiden(andata, resolution = resolution)

    sc.tl.umap(andata)
    
    return lsi_model, resolution

def compute_neighborhood_change(*,control, treatment, cluster_key='leiden', mask_cluster = True):
    
    a1_distances = cosine_similarity(control.obsm['X_lsi'])
    a2_distances = cosine_similarity(treatment.obsm['X_lsi'])
    
    if mask_cluster:
        clusters = control.obs[cluster_key].values
        cluster_square = np.tile(clusters, (len(clusters), 1))

        same_cluster_mask = cluster_square == cluster_square.T
    else:
        same_cluster_mask = np.full(a1_distances.shape, True)
    
    a1_distances = normalize(np.where(same_cluster_mask, 0.0, a1_distances), 'l2')
    a2_distances = normalize(np.where(same_cluster_mask, 0.0, a2_distances), 'l2')
    
    neighborhood_change = 1 - np.sum(np.multiply(a1_distances, a2_distances), axis = 1)
    
    control.obs['neighbor_change'] = neighborhood_change


def make_proportion_plot(*,data,x,y,hue, hue_order = None, order = None, figsize=(5,5)):
    
    if y is None:
        y = '__count'
        data['__count'] = 1
        
    factor_counts = data.groupby(x)[y].sum()
    
    if hue_order is None:
        hue_order = sorted(np.unique(data[hue]))
        
    if order is None:
        order = sorted(np.unique(data[x]))
        
    cumm_prop = {x : 0 for x in order}
    
    groups = data.groupby([x,hue])
    
    new_df = []
    for y_level in hue_order:
        for x_level in order:
            level_count = groups.get_group((x_level, y_level))[y].sum()
            level_prop = cumm_prop[x_level] + level_count/factor_counts[x_level]
            cumm_prop[x_level] = level_prop
            new_df.append((x_level, y_level, level_prop))
            
    new_df = pd.DataFrame(new_df, columns = [x,hue,'proportion'])
    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    sns.barplot(data = new_df, x=x, y='proportion',hue =hue, 
                               hue_order= reversed(hue_order), dodge=False, orient='v',
                               palette=sns.color_palette('hls'), linewidth=0.0, ax= ax)
    ax.legend(loc='upper center', bbox_to_anchor=(1.4, 0.8), shadow=False, ncol=1)
    sns.despine()
    
    return ax

In [3]:
paths = {}
try:
    snakemake
    output_paths,paths = {},{}
    dataset_strlen = len(snakemake.wildcards.dataset)
    for name, path in snakemake.input.items():
        output_paths[name] = '.' + path[dataset_strlen:]
        paths[name] = path
        
    print('When opening this notebook from dataset directory, path to pickle:')
    print('.' + snakemake.output.paths[dataset_strlen:])
    with open(snakemake.output.paths, 'wb') as f:
        pickle.dump(output_paths, f)
except NameError:
    with open('./original_analysis/paths.pkl','rb') as f:
        paths = pickle.load(f)

# Load Data

In [4]:
paths

In [5]:
raw_data = anndata.read_h5ad(paths['counts'])
corrected_data = anndata.read_h5ad(paths['corrected_counts'])
dedup_data = anndata.read_h5ad(paths['dedup_counts'])

dupcounts = pd.read_csv(paths['dupcounts'], sep = '\t')
sample_fragments = read_bias_file(paths['sample_fragments'])

raw_data_lsi_model, resolution = process_counts(raw_data, tune = True)
corrected_data_lsi_model, _ = process_counts(corrected_data, 10**resolution)
dedup_lsi_model, _ = process_counts(dedup_data, 10**resolution)

# Fragment Model Performance

In [6]:
points = benchmark_fragment_model(sample_fragments, dupcounts, raw_data.var)

ax = sns.displot(points, x = 'log_duprate', y = 'true_log_duprate', kind = 'kde')
ax.set(xlabel = 'Predicted Fragment Count', ylabel = 'Observed Fragment Count')

# Compare UMAPs

In [7]:
hue_order = sorted(np.unique(raw_data.obs.leiden))

fig, ax = plt.subplots(1,3, figsize = (14,5))
plot_umap(corrected_data, color_key=raw_data.obs.leiden, quantitative=False, ax = ax[0], hue_order=hue_order)
ax[0].set(title = 'Bias-corrected UMAP', xlabel = 'Colored by UNcorrected clustering')

plot_umap(dedup_data, color_key=raw_data.obs.leiden, quantitative=False, ax = ax[1], legend = False, hue_order=hue_order)
ax[1].set(title = 'Deduplicated UMAP',xlabel = 'Colored by UNcorrected clustering')

plot_umap(raw_data, color_key=raw_data.obs.leiden, quantitative=False, ax = ax[2], legend = True,
         hue_order=hue_order)
ax[2].set(title = 'Raw Count UMAP')
plt.show()

# Observe Bias Concentration

In [8]:
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax = sns.scatterplot(data = raw_data.obs, x = 'fragment_count', y ='mean_log_duprate', hue = 'leiden', legend = False,
                    hue_order= hue_order, linewidth = 0, s = 10, alpha = 0.8, ax = ax)
ax.set(xscale = 'log', xlabel = 'log(Fragment Count)', ylabel = 'Mean log(duplication rate) per cell')
sns.despine()

In [9]:
fig, ax = plt.subplots(1,2, figsize=(14,5))

plot_umap(raw_data, color_key = 'mean_log_duprate', ax = ax[0])
ax[0].set(title = 'Corrected UMAP, Mean Bias Per Cell')
plot_umap(raw_data, color_key='fragment_count', ax = ax[1])
ax[1].set(title = 'Normal UMAP, Mean Bias Per Cell')

# Peak-Bias Heterogeneity

In [10]:
results = []
for i, l in enumerate(raw_data.var.dropna().sample(20).samples.values):
    for x in l.strip('[]').split(', '):
        results.append((i, float(x)))

peak_samples = pd.DataFrame(results, columns = ['peak_id','log_duprate'])

fig, ax = plt.subplots(figsize = (15,5))
sns.violinplot(data = peak_samples, x = 'peak_id', y ='log_duprate', ax = ax)
ax.set(xticks = [], ylabel = 'Log(duprate)', xlabel = 'Peaks')
sns.despine()
plt.show()

# Enrichment of Biased Peaks Per Cluster

In [11]:
cluster_enrichments = get_bias_peak_enrichment(raw_data, 'leiden')

ax = sns.displot(data = cluster_enrichments, hue = 'cluster_id', x = 'rank', 
            kind = 'kde', common_norm = False, hue_order= hue_order, height = 7, aspect = 1.5)
ax.set(xlabel = 'Peak Bias Rank', ylabel = 'Proportion of Fragments')
plt.show()

# Cell-Specific Effects Analysis

In [12]:
unbiased_rank, biased_rank = np.quantile(raw_data.var['rank'].dropna().values, [0.3,0.7])
peaks_of_interest = pd.concat([
    raw_data.var[raw_data.var['rank'] >= biased_rank].sample(10),
    raw_data.var[raw_data.var['rank'] <= unbiased_rank].sample(10)
])['peak_id'].values

peaks_str = '\n'.join(
    peak.replace('_','\t') for peak in peaks_of_interest
)

fragment_file = paths['fragments']

In [13]:
relavant_peaks = !bedtools intersect -a $fragment_file -b stdin -sorted < <(echo "$peaks_str" | sort -k1,1 -k2,2n)

In [14]:
stratified_sample = get_fragment_distribution_by_peak_and_cluster(raw_data, '\n'.join(relavant_peaks), 'leiden')

fig, ax = plt.subplots(figsize = (15,5))
sns.swarmplot(data = stratified_sample.sort_values('rank'), x = 'peak_id', y = 'log_duprate', hue = 'leiden', 
              ax = ax, dodge = True, size = 1.5, 
              order = stratified_sample.groupby('peak_id')['rank'].first().sort_values().index.values)
sns.despine()
ax.set(xticks = [], ylabel = 'Log(Duplication Rate)', xlabel = 'less bias ← Peaks → more bias')
ax.get_legend().remove()
plt.show()

# Differential Peaks Analysis

In [15]:
diffpeaks = get_differential_peaks(raw_data)

In [16]:
for cluster, data in diffpeaks.T.iterrows():
    with open(os.path.join(snakemake.output.diffpeaks, 'cluster_{}_diffpeaks.bed'.format(str(cluster))), 'w') as f:
        print('\n'.join([x.replace('_','\t') for x in data.values]), file = f)

# Delta Diffpeak Comparison

In [17]:
dedup_data.obs['leiden_corrected'] = corrected_data.obs.leiden
sc.tl.rank_genes_groups(corrected_data,'leiden',method='wilcoxon')
sc.tl.rank_genes_groups(dedup_data,'leiden_corrected',method='wilcoxon')

In [18]:
delta_peaks = {}
print('Cluster','Corrected','Deduplicated', sep = '\t')
for cluster in np.unique(corrected_data.obs.leiden):
    diff_corrected = sc.get.rank_genes_groups_df(corrected_data,group=cluster,pval_cutoff=0.05/len(raw_data.obs),
                                                 log2fc_min=1.5).sort_values('pvals')['names'].values
    diff_dedup = sc.get.rank_genes_groups_df(dedup_data, group=cluster,pval_cutoff=0.05/len(raw_data.obs), 
                                             log2fc_min=1.5).sort_values('pvals')['names'].values
    print('{}'.format(cluster),len(diff_corrected),len(diff_dedup), sep = '\t')
    delta_peaks[cluster] = list(set(diff_corrected).difference(set(diff_dedup)))

In [19]:
density = get_bias_peak_enrichment(corrected_data, 'leiden')

ax = sns.displot(data = density, hue = 'cluster_id', x = 'rank', 
            kind = 'kde', common_norm = False, hue_order= hue_order, height = 7, aspect = 1.5)

delta_ranks = pd.DataFrame([(cluster,peak_rank) for cluster, peak_ranks in delta_peaks.items() for peak_rank in peak_ranks], columns = ['cluster','rank'])
sns.rugplot(data = delta_ranks, x = 'rank', hue = 'cluster', legend = False, hue_order=hue_order)

ax.set(xlabel = 'Peak Bias Rank', ylabel = 'Proportion of Fragments', xticks = [])
plt.show()

In [20]:
delta_ranks['rank'] = delta_ranks['rank'].astype(np.float32)
delta_ranks = delta_ranks.merge(raw_data.var.drop(columns = ['samples']), on = 'rank')

delta_ranks[['chr','start','end']] = delta_ranks['peak_id'].str.split('_',expand=True)

delta_ranks['strand'] = '*'

delta_ranks[['chr','start','end','strand','rank','median_log_duprate','fragment_count','accessible_in_cells']]\
    .to_csv(snakemake.output.delta_diffpeaks, index=None, sep = '\t')

# Peak Annotation Analysis

In [21]:
annotations = pd.read_csv(paths['peak_annotations'], sep = '\t')
annotations.annotation = np.where(annotations.annotation.str.contains('Intron'), 'Intron', annotations.annotation)
annotations.annotation = np.where(annotations.annotation.str.contains('exon'), 'Exon', annotations.annotation)
annotations['start'] = annotations.start - 1
annotations['peak_id'] = get_peak_id(annotations, keys = ['seqnames','start','end'])

annotations = annotations.merge(raw_data.var[['peak_id','median_log_duprate']], on = 'peak_id')
annotations = annotations.rename(columns = {'V4' : "TAD"}).drop(columns=['V5','V6'])
annotations['bias_bin'], bins = pd.qcut(annotations.median_log_duprate, 3, labels = ['Least Bias','Average','Most Bias'], retbins=True)

In [22]:
feature_order = ['Promoter (<=1kb)', 'Promoter (1-2kb)', 'Promoter (2-3kb)',
                 "5' UTR",'Intron','Exon',"3' UTR", 'Downstream (<1kb)', 'Downstream (1-2kb)', 'Downstream (2-3kb)',
                'Distal Intergenic']

### Enrichment of genomic features by bias

In [23]:
make_proportion_plot(data = annotations, hue='annotation',x='bias_bin',y=None,hue_order=feature_order,
                         order = ['Least Bias','Average','Most Bias'], figsize=(5,8))

### Distribution of bias by genomic feature

In [24]:
fig, ax = plt.subplots(1,1,figsize=(10,5))
sns.violinplot(data = annotations, x = 'annotation', y ='median_log_duprate', ax= ax, order=feature_order, palette=reversed(sns.color_palette('hls')))
sns.despine()
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
ax.set(xlabel = 'Peak Annotation',ylabel= 'Median Log Duprate of Peaks')
plt.show()

In [25]:
raw_data.var = raw_data.var.merge(annotations[['peak_id','annotation', 'TAD']], on = 'peak_id')

In [26]:
cell_annotation_matrix = pd.DataFrame(raw_data.X.todense(),
        index = raw_data.obs.leiden.values, columns = raw_data.var.annotation.values)

cell_annotation_matrix = cell_annotation_matrix.reset_index().melt(id_vars = 'index', value_name='count',var_name='annotation')
cell_annotation_matrix = cell_annotation_matrix[cell_annotation_matrix['count'] > 0]

cell_groups = cell_annotation_matrix.groupby(['index','annotation']).sum().reset_index()\
    .rename(columns = {'index' : 'cluster ID'})

In [27]:
cell_groups_matrix = cell_groups.pivot(index = 'cluster ID', columns = 'annotation', values='count')[feature_order]
cell_groups_matrix = cell_groups_matrix / cell_groups_matrix.sum(axis = 0)

normed_matrix = (cell_groups_matrix.T - cell_groups_matrix.T.mean(axis = 0)) / cell_groups_matrix.T.std(axis = 0)

In [28]:
#fig,ax = plt.subplots(1,1,figsize=(6,6))
sns.clustermap(normed_matrix, cmap='vlag', square = True)#, ax = ax)

In [29]:
plot_umap(raw_data, color_key='leiden',quantitative=False, legend=True, hue_order=hue_order)

# Promoter vs. Enhancer vs. TAD Clustering

In [30]:
raw_data.obs.index = raw_data.obs.index.astype(str)
raw_data.var.index = raw_data.var.index.astype(str)
enhancer_peaks = ~raw_data.var.annotation.str.contains('Promoter')
enhancers = raw_data[:, enhancer_peaks]
promoters = raw_data[:, ~enhancer_peaks]

process_counts(enhancers, 10**resolution)
process_counts(promoters, 10**resolution)

In [31]:
tad_domains = raw_data.var.TAD.unique()
raw_data.var.index = raw_data.var.index.astype(str)
tad_matrix = AnnData( X = np.hstack([
    raw_data[:, raw_data.var.TAD == tad].X.sum(axis = 1)
    for tad in tad_domains
]), obs = pd.DataFrame(raw_data.obs.barcode.values, columns = ['barcode']), var = pd.DataFrame(tad_domains, columns = ['TAD']))     
tad_matrix.obs['leiden_raw'] = raw_data.obs.leiden
tad_matrix.var = tad_matrix.var.set_index('TAD')

tad_matrix.raw = tad_matrix
sc.pp.normalize_total(tad_matrix, target_sum=1000)
sc.pp.log1p(tad_matrix)
sc.pp.scale(tad_matrix)
sc.tl.pca(tad_matrix)
sc.pp.neighbors(tad_matrix)
sc.tl.umap(tad_matrix)
sc.tl.leiden(tad_matrix, resolution=10**resolution)
sc.tl.rank_genes_groups(tad_matrix, 'leiden_raw', method='wilcoxon', use_raw=True)

### Different Genomic Resolutions for Clustering

In [32]:
fig, ax = plt.subplots(1,3,figsize=(14,4))
plot_umap(enhancers, color_key = raw_data.obs.leiden, quantitative=False, ax = ax[0], hue_order=hue_order)
plot_umap(promoters, color_key = raw_data.obs.leiden, quantitative=False, ax = ax[1], hue_order=hue_order)
plot_umap(tad_matrix, color_key= raw_data.obs.leiden, quantitative=False, ax = ax[2], hue_order=hue_order)
for i, title in zip(range(3), ['Enhancers','Promoters','TADs']):
    ax[i].set(title= title)

### Which cells experience the greatest change in neighborhood, Enhancers vs Promoters?

In [33]:
compute_neighborhood_change(control=enhancers, treatment=promoters, cluster_key='leiden')

ax = plot_umap(enhancers, color_key='neighbor_change')
ax.set(title = 'Change in neighbors due to treatment')
plt.show()

### Which types of TADs are discriminative between cell types?

In [34]:
diff_tads = sc.get.rank_genes_groups_df(tad_matrix, '0')
tad_activity = raw_data.var.groupby('TAD')['accessible_in_cells'].mean()
diff_tads = diff_tads.merge(tad_activity, left_on='names', right_on='TAD')
diff_tads['log_10_p'] = - np.log10(diff_tads.pvals)

sns.scatterplot(data = diff_tads, x = 'logfoldchanges', y = 'log_10_p', hue = 'accessible_in_cells')
sns.despine()

# Doublets

In [35]:
doublet_data = pd.read_csv(paths['doublets'], sep = '\t', header = None)
raw_data.obs[['doublet_score','is_doublet']] = doublet_data.values

In [37]:
fig, ax = plt.subplots(1,2,figsize=(12,4))
data = raw_data.obsm['X_umap']
sns.scatterplot(x = data[:,0], y = data[:,1], hue = standardize(raw_data.obs.doublet_score),
           palette = "vlag", size = ~raw_data.obs.is_doublet, legend = False, ax = ax[0])
ax[0].set(xticklabels = [], xticks = [], yticks = [])
sns.despine()
plot_umap(raw_data, color_key = raw_data.obs.leiden, quantitative=False, ax = ax[1], hue_order=hue_order)
ax[1].set(title = 'Enhancer Clustering, Control Clustering')

In [38]:
fig,ax = plt.subplots(1,2,figsize=(12,4))
plot_umap(enhancers, color_key=raw_data.obs.doublet_score, ax = ax[0])
ax[0].set(title = 'UMAP with Enhancers Only, colored by doublet score')
plot_umap(promoters, color_key=raw_data.obs.doublet_score, ax = ax[1])
ax[1].set(title = 'UMAP with Promoters Only, colored by doublet score')

In [39]:
enhancers.obs['fragment_count'] = enhancers.X.sum(axis = 1)
sns.scatterplot(x= enhancers.obs.fragment_count, y = raw_data.obs.doublet_score, hue = raw_data.obs.is_doublet, 
               palette='vlag')
sns.despine()